AkHQ: http://localhost:8082  
Spark-Master http://localhost:8083  
Spark-Worker-1 http://localhost:8084

https://github.com/aehrc/pathling/tree/issue/452/lib/python#python-api-for-pathling

# Try Connect To Test FHIR Kafka

In [1]:
!python --version
!cat /usr/local/bin/before-notebook.d/spark-config.sh

Python 3.9.12
#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# included in all the spark scripts with source command
# should not be executable directly
# also should not be passed any arguments, since we need original $*

# symlink and absolute path should rely

In [2]:
import os
os.environ

environ{'SHELL': '/bin/bash',
        'HOSTNAME': '08b42fc39e7e',
        'LANGUAGE': 'en_US.UTF-8',
        'SPARK_OPTS': '--driver-java-options=-Xms1024M --driver-java-options=-Xmx4096M --driver-java-options=-Dlog4j.logLevel=info',
        'NB_UID': '1000',
        'PWD': '/home/jovyan',
        'PYSPARK_SUBMIT_ARGS': '     --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,au.csiro.pathling:encoders:5.0.3-SNAPSHOT     --repositories https://oss.sonatype.org/content/repositories/snapshots     pyspark-shell',
        'HOME': '/home/jovyan',
        'LANG': 'en_US.UTF-8',
        'NB_GID': '100',
        'XDG_CACHE_HOME': '/home/jovyan/.cache/',
        'APACHE_SPARK_VERSION': '3.2.1',
        'PYTHONPATH': '/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip:/usr/local/spark/python:',
        'HADOOP_VERSION': '3.2',
        'SHLVL': '0',
        'CONDA_DIR': '/opt/conda',
        'SPARK_HOME': '/usr/local/spark',
        'SPARK_CONF_DIR': '/usr/local/spark/conf',
        'NB_U

In [3]:
import findspark
findspark.init()
findspark.find()

'/usr/local/spark'

In [2]:
appName = "Kafka, Spark and FHIR Data"
master = "spark://spark-master:7077"
#master = "local[*]"
kafka_topic = "fhir.post-gateway-kdb"

In [3]:
from pyspark.sql import SparkSession
from pathling.etc import find_jar

#spark = SparkSession.builder \
#        .appName(appName) \
#        .master(master) \
#        .config('spark.jars', find_jar()) \
#        .config('spark.ui.port','0') \
#        .getOrCreate()

spark = SparkSession.builder \
        .appName(appName) \
        .master(master) \
        .config('spark.ui.port','0') \
        .getOrCreate()

In [4]:
print(spark.sparkContext.getConf().getAll())

[('spark.repl.local.jars', 'file:///home/jovyan/.ivy2/jars/org.apache.spark_spark-sql-kafka-0-10_2.12-3.2.1.jar,file:///home/jovyan/.ivy2/jars/au.csiro.pathling_encoders-5.0.3-SNAPSHOT.jar,file:///home/jovyan/.ivy2/jars/org.apache.spark_spark-token-provider-kafka-0-10_2.12-3.2.1.jar,file:///home/jovyan/.ivy2/jars/org.apache.kafka_kafka-clients-2.8.0.jar,file:///home/jovyan/.ivy2/jars/com.google.code.findbugs_jsr305-3.0.0.jar,file:///home/jovyan/.ivy2/jars/org.apache.commons_commons-pool2-2.6.2.jar,file:///home/jovyan/.ivy2/jars/org.spark-project.spark_unused-1.0.0.jar,file:///home/jovyan/.ivy2/jars/org.apache.hadoop_hadoop-client-runtime-3.3.1.jar,file:///home/jovyan/.ivy2/jars/org.lz4_lz4-java-1.7.1.jar,file:///home/jovyan/.ivy2/jars/org.xerial.snappy_snappy-java-1.1.8.4.jar,file:///home/jovyan/.ivy2/jars/org.apache.hadoop_hadoop-client-api-3.3.1.jar,file:///home/jovyan/.ivy2/jars/org.apache.htrace_htrace-core4-4.1.0-incubating.jar,file:///home/jovyan/.ivy2/jars/commons-logging_common

In [5]:
# https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html
# default for startingOffsets is "latest", but "earliest" allows rewind for missed alerts    
df = spark \
  .readStream  \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka1:19092") \
  .option("subscribe", kafka_topic) \
  .option("startingOffsets", "earliest") \
  .load()

In [6]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
import os
os.getenv("PYSPARK_SUBMIT_ARGS")

'     --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,au.csiro.pathling:encoders:5.0.3-SNAPSHOT     --repositories https://oss.sonatype.org/content/repositories/snapshots     pyspark-shell'

In [8]:
query = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
          .writeStream \
          .queryName("gettable") \
          .format("memory") \
          .start()

# close connection after 30 seconds
query.awaitTermination(30)

False

In [9]:
kafka_data = spark.sql("select * from gettable")
kafka_data.show()
type(kafka_data)

+-------------------+--------------------+
|                key|               value|
+-------------------+--------------------+
|    Patient/patient|{"resourceType": ...|
|Condition/condition|{"resourceType": ...|
|Condition/condition|{"resourceType": ...|
|Procedure/procedure|{"resourceType": ...|
|Encounter/encounter|{"resourceType": ...|
+-------------------+--------------------+



pyspark.sql.dataframe.DataFrame

In [10]:
pd_df = kafka_data.toPandas()
pd_df

,key,value
0,Patient/patient,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
1,Condition/condition,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
2,Condition/condition,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
3,Procedure/procedure,"{""resourceType"": ""Bundle"",""meta"": {""security"":..."
4,Encounter/encounter,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."


# Bring Pathling into the game

In [11]:
from pathling.r4 import bundles

resources = bundles.from_json(kafka_data, 'value')

In [12]:
patients = bundles.extract_entry(spark, resources, 'Patient')
encounter = bundles.extract_entry(spark, resources, 'Encounter')
condition = bundles.extract_entry(spark, resources, 'Condition')

In [13]:
patients.printSchema()

root
 |-- id: string (nullable = true)
 |-- id_versioned: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- versionId: string (nullable = true)
 |    |-- versionId_versioned: string (nullable = true)
 |    |-- lastUpdated: timestamp (nullable = true)
 |    |-- source: string (nullable = true)
 |    |-- profile: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- security: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- system: string (nullable = true)
 |    |    |    |-- version: string (nullable = true)
 |    |    |    |-- code: string (nullable = true)
 |    |    |    |-- display: string (nullable = true)
 |    |    |    |-- userSelected: boolean (nullable = true)
 |    |-- tag: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable

In [14]:
patients.select("id", "birthDate", "gender", "address.postalCode").toPandas()

,id,birthDate,gender,postalCode
0,patient,1990-01-01,female,[12345]


In [15]:
encounter.printSchema()

root
 |-- id: string (nullable = true)
 |-- id_versioned: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- versionId: string (nullable = true)
 |    |-- versionId_versioned: string (nullable = true)
 |    |-- lastUpdated: timestamp (nullable = true)
 |    |-- source: string (nullable = true)
 |    |-- profile: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- security: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- system: string (nullable = true)
 |    |    |    |-- version: string (nullable = true)
 |    |    |    |-- code: string (nullable = true)
 |    |    |    |-- display: string (nullable = true)
 |    |    |    |-- userSelected: boolean (nullable = true)
 |    |-- tag: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable

In [16]:
encounter.select("id", "subject.reference", "serviceType.coding.code", "period.start", "period.end").toPandas()

,id,reference,code,start,end
0,encounter,Patient/patient,"[2200, 2200]",2021-11-09T05:41:53+00:00,None


In [17]:
condition.printSchema()

root
 |-- id: string (nullable = true)
 |-- id_versioned: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- versionId: string (nullable = true)
 |    |-- versionId_versioned: string (nullable = true)
 |    |-- lastUpdated: timestamp (nullable = true)
 |    |-- source: string (nullable = true)
 |    |-- profile: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- security: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- system: string (nullable = true)
 |    |    |    |-- version: string (nullable = true)
 |    |    |    |-- code: string (nullable = true)
 |    |    |    |-- display: string (nullable = true)
 |    |    |    |-- userSelected: boolean (nullable = true)
 |    |-- tag: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable

In [18]:
condition.select("id", "encounter.reference", "code.coding.code").toPandas()

,id,reference,code
0,condition,Encounter/encounter,[H57.0]
1,condition,Encounter/encounter,[H57.0]
